In [4]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from glob import glob

In [5]:
files = glob('url/*.json')
len(files)

4490

In [6]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        urls.extend(json.load(fopen))

100%|████████████████████████████████████| 4490/4490 [00:00<00:00, 66877.70it/s]


In [7]:
# !rm -rf page
!mkdir page

In [8]:
urls = sorted(list(set(urls)))
len(urls)

101119

In [9]:
urls[0]

'https://www.malaysia-today.net/2004/11/23/anwars-open-house-attracted-malaysians-from-all-walks-of-life/'

In [20]:
r = requests.get('https://www.malaysia-today.net/2023/06/13/serahan-dokumen-kes-seks-bekas-pengarah-keselamatan-bersatu-17-julai/')
soup = BeautifulSoup(r.content, "lxml")

In [27]:
p = soup.find('div', {'class': 'entry-content'}).find_all('p')
p = [p_.text for p_ in p]
p

['',
 '(Berita Harian) – Hakim, Siti Aminah Ghazali, memutuskan tarikh itu ketika sebutan kes yang turut dihadiri peguam bela Wan Mohammad Ashraf, Datuk Ishak Mohd Yusoff dan Datuk Mohamad Hanafiah Zakaria, serta Timbalan Pendakwa Raya, Nidzuwan Abd Latip.',
 'Terdahulu, Nidzuwan memohon satu tarikh sebutan kes baharu untuk serahan baki dokumen kepada pembelaan.',
 '“Kita ada serahkan dokumen kes itu hari ini kepada pihak pembelaan, bakinya akan diserahkan pada sebutan kes seterusnya,” katanya.',
 'Susulan itu, Mohamad Hanafiah mengesahkan menerima dokumen terbabit hari ini.',
 'Pada 11 Mei lalu, Wan Mohammad Ashraf yang juga ahli perniagaan mengaku tidak bersalah atas pertuduhan melakukan seks menggunakan objek terhadap pegawai bank tanpa kebenaran wanita itu.',
 'Wan Mohammad Ashraf, 42, didakwa melakukan perbuatan itu terhadap mangsa berusia 26 tahun di tandas wanita sebuah restoran di hotel di Jalan Bukit Bintang di sini pada jam 5 pagi, 2 Januari lalu.',
 'Pertuduhan mengikut Seks

In [28]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        title = soup.find('h1', {'class': 'single-post-title'}).text.strip()
    except:
        title = None
    try:
        h4 = soup.find_all('h4')
        h4 = [h4_.text for h4_ in h4]
    except:
        h4 = None
    try:
        p = soup.find('div', {'class': 'entry-content'}).find_all('p')
        p = [p_.text for p_ in p]
    except:
        p = None
    
    data = {
        'url': url,
        'title': title,
        'h4': h4,
        'p': p,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [30]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls_ = [(urls[a], os.path.join('page', f'{a}.json')) for a in aranged]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

    for future in as_completed(futures):
        future.result()

100%|██████████████████████████████████▉| 10111/10112 [5:02:31<00:01,  1.80s/it]


IndexError: list index out of range